In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Wczytywanie danych i zamiana danych kategorycznych
df = pd.read_csv("C:/Users/Dominika/Desktop/IiE/Uczenie maszynowe/stroke.csv", sep=';')

df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df['Residence_type'] = df['Residence_type'].map({'Urban': 1, 'Rural': 0})
df = pd.get_dummies(df, columns=['work_type', 'smoking_status'], drop_first=True)
df = df.astype(int, errors='ignore')

print(df.head())

   gender   age  hypertension  heart_disease  ever_married  Residence_type  \
0       1  67.0             0              1             1               1   
1       0  61.0             0              0             1               0   
2       1  80.0             0              1             1               0   
3       0  49.0             0              0             1               1   
4       0  79.0             1              0             1               0   

   avg_glucose_level   bmi  stroke  work_type_Never_worked  work_type_Private  \
0             228.69  36.6       1                       0                  1   
1             202.21   NaN       1                       0                  0   
2             105.92  32.5       1                       0                  1   
3             171.23  34.4       1                       0                  1   
4             174.12  24.0       1                       0                  0   

   work_type_Self-employed  work_type_childr

In [3]:
# Usuwanie danych brakujących
print(df.isna().sum())

df_clean = df.dropna()
print(df_clean.shape)

gender                              0
age                                 0
hypertension                        0
heart_disease                       0
ever_married                        0
Residence_type                      0
avg_glucose_level                   0
bmi                               201
stroke                              0
work_type_Never_worked              0
work_type_Private                   0
work_type_Self-employed             0
work_type_children                  0
smoking_status_formerly smoked      0
smoking_status_never smoked         0
smoking_status_smokes               0
dtype: int64
(4908, 16)


In [4]:
print(df['stroke'].value_counts())

# metoda SMOTE (tworzenie sztucznych próbek mniejszościowych)
from sklearn.utils import resample

df_majority = df_clean[df.stroke == 0]
df_minority = df_clean[df.stroke == 1]

df_minority_upsampled = resample(
    df_minority,
    replace=True,              
    n_samples=len(df_majority),
    random_state=42
)

df_balanced = pd.concat([df_majority, df_minority_upsampled])

print(df_balanced['stroke'].value_counts())


stroke
0    4860
1     249
Name: count, dtype: int64
stroke
0    4699
1    4699
Name: count, dtype: int64


C:\Users\Dominika\AppData\Local\Temp\ipykernel_20244\713573422.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_majority = df_clean[df.stroke == 0]
C:\Users\Dominika\AppData\Local\Temp\ipykernel_20244\713573422.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_minority = df_clean[df.stroke == 1]


In [5]:
X = df_balanced.drop('stroke', axis=1)
y = df_balanced['stroke']

# Podział na zbiór treningowy i testowy i standaryzacja 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Pytania:
#### 1. Czym jest podział danych na zbiór uczący i testowy? W jakim celu się go wykonuje? W jaki sposób powinien zostać wykonany?
Zbiór danych uczący (training set) służy do trenowania modelu, w tym nauki wzorców i zależności między zmiennymi. Zbiór testowy (test set) wykorzystuje się do oceny jakości modelu po zakończeniu treningu danych. Podział danych wykorzystuje się, aby sprawdzić czy model działa poprawnie zarówno na danych, które znał, jak i na danych nowych. Dodatkowo stosuje się podzaiał, żeby unikać przeuczenia, które może sztucznie zawyżać wyniki. Podział pozwala wykryć czy model został poprawnie wykonany i "rozumie" dane. Podział powinien zostać wykonany losowo, tak aby każdy przypadek miał szansę trafić do którejkolwiek części. Proporcje klas powinny zostać jednka zachowane, tak żeby były takie same w obu zbiorach. Dane również powinny być podzielone prawidłowo ze względu na proporcję. Najczęściej spotyka się podziały 70% zbiór uczący i 30% zbiór testowy, a także proporcje 80/20 lub 60/40.
#### 2. Czym jest macierz błędu?
Macierz błędu to narzędzie wykorzystywane do oceny jakości modelu klasyfikacyjnego. Macierz pokazuje ile przypadków model sklasyfikował poprawnie, a ile błędnie. Najczęściej ma postać 2x2 i mówi czy model: TP: poprawnie przewidział klasę pozytywną, FP: błędnie uznał przypadek negatywny za pozytywny, FN: nie wykrył pozytywnego przypadku, TN: model poprawnie wykrył klasę negatywną.

#### 3. Miary jakości klasyfikacji - dokładność, czułość i specyficzność - czym się charakteryzują? Która z tych miar jest najważniejsza. Jakie są inne miary jakości klasyfikacji? 
Dokładność odpowiada na pytanie, jaki odsetek wszystkich przypadków został poprawnie sklasyfikowany. Jest ona łatwa w interpretacji i sprawdza się, gdy klasy są zrównoważone. Czułość, pokazuje jaki procent rzeczywistych przypadków pozytywnych wykrył model. Jest ona bardzo ważna gdy rozpatrujemy przypadek, gdzie przeoczenie pozytywnego przypadku ma poważne konsekwencje. Wysoka czułość może oznaczać wiele błędnie uznanych przypadków negatywnych za pozytywne. Specyficzność, opisuje jaki procent przypadków negatywnych zostanie poprawnie rozpoznany. Sama w sobie nie pokazuje, jak model wykrywa pozytywne przypadki. Nie możemy powiedzieć, że jedna miara jest najważniejsza, trzeba patrzeć na kilka na raz. Ich ważność zmienia się jednak w zależności od kontekstu problemu, na przykład czułość sprawdza się lepiej w medycynie, a dokładność gdy mamy zrównoważone dane. Innymi miarami klasyfikacji są:
- prezycja, która opisuje jaki procent przewidzianych przypadków był naprawdę pozytywny
- F1-score, czyli średnia harmoniczna precyzji i czułości
- ROC Curve / AUC, ocenia zdolności modelu do rozróżnienia klas
   

In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Trening i ewaluacja modelu KNN dla różnych wartości k
for k in range(3, 14):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"k={k}: dokładność={accuracy:.4f}, czułość={sensitivity:.4f}, specyficzność={specificity:.4f}")

k=3: dokładność=0.9532, czułość=1.0000, specyficzność=0.9081
k=4: dokładność=0.9532, czułość=1.0000, specyficzność=0.9081
k=5: dokładność=0.9340, czułość=1.0000, specyficzność=0.8706
k=6: dokładność=0.9340, czułość=1.0000, specyficzność=0.8706
k=7: dokładność=0.9213, czułość=1.0000, specyficzność=0.8455
k=8: dokładność=0.9213, czułość=1.0000, specyficzność=0.8455
k=9: dokładność=0.9043, czułość=1.0000, specyficzność=0.8121
k=10: dokładność=0.9043, czułość=1.0000, specyficzność=0.8121
k=11: dokładność=0.8947, czułość=1.0000, specyficzność=0.7933
k=12: dokładność=0.8947, czułość=1.0000, specyficzność=0.7933
k=13: dokładność=0.8830, czułość=1.0000, specyficzność=0.7704


In [7]:
# Trenowanie modelu KNN dla k=3
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [16]:
y_train_pred_knn = knn.predict(X_train_scaled) 
y_test_pred_knn = knn.predict(X_test_scaled)

# Macierze błędów
print(confusion_matrix(y_train_pred_knn, y_train))

[[4047    0]
 [ 173 4238]]


In [9]:
print(confusion_matrix(y_test_pred_knn, y_test)) 

[[435   0]
 [ 44 461]]


# Pytania:
#### 1. Jak działa metoda k-najbliższych sąsiadów?
Metoda k-najbliższych sąsiadów polega na tym, że dla nowego punktu model szuka k najbliższych obserwacji w zbiorze uczącym i następnie przypisuje klasę, która najczęściej występuje wśród tych sąsiadów. 
#### 2. Czym jest k w metodzie KNN? Jak dobrać odpowiednią wartość k? Czy k powinno być liczbą parzystą, nieparzystą, czy nie ma to znaczenia?
K jest to liczba najbliższych sąsiadów, którzy biorą udział w decyzji o klasyfikacji nowego punktu. Najczęściej k wybiera się poprzez testowanie różnych wartości i wybranie tej z najlepszą skutecznością. Przy klasyfikacji binarnej lepszym wyborem jest nieparzyste k, aby uniknąć remisów. W przypadkach gdzie mamy więcej niż 2 klasy parzystość k nie ma aż takiego znaczenia.
#### 3. Czy standaryzacja danych jest wymagana w przypadku wykorzystywania metody k-najbliższych sąsiadów?Dlaczego tak/nie?
Tak, w metodnie k-NN standaryzacja jest wymagana, aby wszystkie zmienne miały równy wpływ na decyzję modelu, ponieważ gdy zmienne mają różne skale, to te o większych wartościach liczbowych będą dominować i zniekształcać wyniki.
#### 4. Czy wielkość zbioru danych ma znaczenie w przypadku tej metody?
Wielkość zbioru danych ma duże znaczenie w metodzie k-NN. Metoda ta dobrze radzi sobie przy małych lub średnich zbiorach danych, natomiast przy bardzo dużych staje się wolna i nieefektywna.
#### 5. W jaki sposób metoda KNN może zostać wykorzystana do rozwiązania problemu regresji?
W regresji metoda k-NN przewiduje wartość liczbową, obliczając średnią lub ważoną średnią wartości zmiennej objaśniającej. Dzięki temu nowa obserwacja otrzymuje wynik oparty na podobnych przypadkach danych uczących.
#### 6. Czy w metodzie tej można wykorzystać i jeśli tak, to w jaki sposób zmienne kategoryczne?
Tak, można wykorzystac zmienne kategoryczne, jednak trzeba przekształcić je tak, aby k-NN mogło je uwzględnić w klasyfikacji. Można wykorzystać do tego na przykład kodowanie one-hot, gdzie każda kategoria zamieniona jest na wektor binarny, który da się uwzględnić w obliczaniu odległości.

In [10]:
for k in range(3, 14):
    kknn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    kknn.fit(X_train_scaled, y_train)
    y_pred = kknn.predict(X_test_scaled)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    accuracy = accuracy_score(y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"k={k}: dokładność={accuracy:.4f}, czułość={sensitivity:.4f}, specyficzność={specificity:.4f}")

k=3: dokładność=0.9553, czułość=1.0000, specyficzność=0.9123
k=4: dokładność=0.9532, czułość=1.0000, specyficzność=0.9081
k=5: dokładność=0.9404, czułość=1.0000, specyficzność=0.8831
k=6: dokładność=0.9372, czułość=1.0000, specyficzność=0.8768
k=7: dokładność=0.9298, czułość=1.0000, specyficzność=0.8622
k=8: dokładność=0.9266, czułość=1.0000, specyficzność=0.8559
k=9: dokładność=0.9213, czułość=1.0000, specyficzność=0.8455
k=10: dokładność=0.9160, czułość=1.0000, specyficzność=0.8351
k=11: dokładność=0.9117, czułość=1.0000, specyficzność=0.8267
k=12: dokładność=0.9053, czułość=1.0000, specyficzność=0.8142
k=13: dokładność=0.9000, czułość=1.0000, specyficzność=0.8038


In [11]:
# Trenowanie modelu KKNN dla k=3
weighted_knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
weighted_knn.fit(X_train_scaled, y_train)

,n_neighbors,3
,weights,'distance'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [ ]:
y_train_pred_weighted_knn = weighted_knn.predict(X_train_scaled)
y_test_pred_weighted_knn = weighted_knn.predict(X_test_scaled) 

# Macierze błędów
print(confusion_matrix(y_train_pred_weighted_knn, y_train)) 

[[4220    0]
 [   0 4238]]


In [13]:
print(confusion_matrix(y_test_pred_weighted_knn, y_test)) 

[[437   0]
 [ 42 461]]


In [14]:
#Porównanie wyników KNN i KKNN:
print("KNN Classification Report:")
print(classification_report(y_test, y_test_pred_knn))           
print("KKNN Classification Report:")
print(classification_report(y_test, y_test_pred_weighted_knn))
        

KNN Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       479
           1       0.91      1.00      0.95       461

    accuracy                           0.95       940
   macro avg       0.96      0.95      0.95       940
weighted avg       0.96      0.95      0.95       940

KKNN Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       479
           1       0.92      1.00      0.96       461

    accuracy                           0.96       940
   macro avg       0.96      0.96      0.96       940
weighted avg       0.96      0.96      0.96       940



In [30]:
# Model KKNN jest nieznacznie lepszy, ponieważ osiąga wyższą dokładność i nieco lepsze F1-score

# Pytania:
#### 1. Co odróżnia metodę KKNN od metody KNN?
W metodzie KNN każdy z sąsiadów ma taki sam wpływ na decyzję klasyfikacji, natomiast w metodzie KKNN każdy sąsiad ma przypisaną wagę zależną od odległości.
#### 2. Czy w metodzie tej występują ograniczenia dotyczące wyboru wartości k?
Metoda KKNN jest bardziej odporna na szum przy małych k i rzadziej daje remisy, ale wartość k nadal należy dobierać poprzez testowanie. 
#### 3. Czy wielkość zbioru danych ma znaczenie w przypadku tej metody? 
Tak samo jak w metodzie KNN, metoda ta najlepiej działa na małych i średnich zbiorach danych, a przy dużych warto rozważyć metody przyśpieszające wyszukiwanie sąsiadów.